## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns= ["EIN","NAME"],inplace=True)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# Please Note: cutoff value is 528

application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
class_types = application_df['CLASSIFICATION'].value_counts()
class_types

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:

# You may find it helpful to look at CLASSIFICATION value counts >1
class_types.loc[class_types>1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# Note cutoff value is 1883
classifications_to_replace = class_types.iloc[5:].index.to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df)
application_df_dummies.head(2)


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False


In [10]:
# Split our preprocessed data into our features and target arrays

# identifying target array "y". This will be the IS_SUCCESSFUL COLUMN
y = application_df_dummies['IS_SUCCESSFUL'].values

# identifying features array "X". 
X = application_df_dummies.drop(columns=['IS_SUCCESSFUL'],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

### First attempt - 73% accuracy

Using two input layers with reLU activation function 

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  6
hidden_nodes_layer2 = 6

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

c:\Users\15199\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 6)              │           264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │            42 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 313 (1.22 KB)

 Trainable params: 313 (1.22 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [14]:
# Train the model (ATTEMPT #1)
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 518us/step - accuracy: 0.6268 - loss: 0.6605
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 503us/step - accuracy: 0.7214 - loss: 0.5711
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step - accuracy: 0.7229 - loss: 0.5688
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 462us/step - accuracy: 0.7270 - loss: 0.5594
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 499us/step - accuracy: 0.7229 - loss: 0.5632
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 501us/step - accuracy: 0.7284 - loss: 0.5570
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 453us/step - accuracy: 0.7279 - loss: 0.5574
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 460us/step - accuracy: 0.7343 - loss: 0.5492
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 489us/step - accuracy: 0.7324 - loss: 0.5511
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 490us/step - accuracy: 0.7284 - loss: 0.5571
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 487us/step - accuracy: 0.7287 - loss: 0.5560
Epoch 12/100
804/80

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 669us/step - accuracy: 0.7237 - loss: 0.5556
Loss: 0.5555906295776367, Accuracy: 0.7237317562103271


In [16]:
# Export our model to HDF5 file
filepath = r"C:\Users\15199\OneDrive\Documentos\UofT Data Analytics BootCamp\Module Challenges\Module 21\deep-learning-challenge\AlphabetSoupCharity.h5"
nn.save(filepath, save_format='h5')

### Second Attempt

Conditions: Adding additional units to each hidden layer and on more hidden layer

Results: 73% accuracy, a 1% increase



In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer =  10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 10)             │           440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 671 (2.62 KB)

 Trainable params: 671 (2.62 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model (ATTEMPT #2)
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 557us/step - accuracy: 0.6028 - loss: 0.6500
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step - accuracy: 0.7233 - loss: 0.5704
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step - accuracy: 0.7258 - loss: 0.5602
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 502us/step - accuracy: 0.7325 - loss: 0.5539
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 509us/step - accuracy: 0.7327 - loss: 0.5533
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 499us/step - accuracy: 0.7294 - loss: 0.5567
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 522us/step - accuracy: 0.7329 - loss: 0.5472
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 497us/step - accuracy: 0.7317 - loss: 0.5520
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 501us/step - accuracy: 0.7372 - loss: 0.5471
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step - accuracy: 0.7332 - loss: 0.5480
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 490us/step - accuracy: 0.7310 - loss: 0.5518
Epoch 12/100
804/80

KeyboardInterrupt: 

In [ ]:
# Evaluate the model using the test data (Attempt #2)
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 423us/step - accuracy: 0.7265 - loss: 0.5539
Loss: 0.5539145469665527, Accuracy: 0.7265306115150452


### Third Attempt

Conditions: Changing the activation function of the first hidden layer to leakyReLu. Updating the no of neurons in 1st layer to be 2/3 of number of input unites. See source https://www.kaggle.com/discussions/general/321114

Results: Accuracy 73% - AKA no change from previous attempt

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer_1st =  31
hidden_nodes_layer = 10 

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer_1st, activation="leaky_relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 31)             │         1,364 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 10)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,805 (7.05 KB)

 Trainable params: 1,805 (7.05 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model (ATTEMPT #2)
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 573us/step - accuracy: 0.6863 - loss: 0.6045
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 559us/step - accuracy: 0.7331 - loss: 0.5531
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 594us/step - accuracy: 0.7325 - loss: 0.5529
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 554us/step - accuracy: 0.7347 - loss: 0.5506
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 520us/step - accuracy: 0.7296 - loss: 0.5526
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 573us/step - accuracy: 0.7304 - loss: 0.5515
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 530us/step - accuracy: 0.7352 - loss: 0.5445
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 548us/step - accuracy: 0.7301 - loss: 0.5524
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 548us/step - accuracy: 0.7331 - loss: 0.5497
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 589us/step - accuracy: 0.7404 - loss: 0.5414
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 550us/step - accuracy: 0.7365 - loss: 0.5447
Epoch 12/100
804/80

In [24]:
# Evaluate the model using the test data (Attempt #2)
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 699us/step - accuracy: 0.7255 - loss: 0.5512
Loss: 0.5512425899505615, Accuracy: 0.7254810333251953


In [25]:
# Export our model to HDF5 file
filepath = r"C:\Users\15199\OneDrive\Documentos\UofT Data Analytics BootCamp\Module Challenges\Module 21\deep-learning-challenge\AlphabetSoupCharity_Optimization.h5"
nn.save(filepath, save_format='h5')